In [1]:
import pandas as pd

In [6]:
data = pd.read_csv("../Current_Data/New_data.csv", usecols=['Country','Date','Cases','Deaths','Summary'])
data.head()

,Country,Date,Cases,Deaths,Summary
0,Benin,12/30/13,11,0.0,Two districts crossed the alert threshold at t...
1,Burkina Faso,12/30/13,88,13.0,Two districts crossed the alert threshold at t...
2,Côte d'Ivoire,12/30/13,3,2.0,Two districts crossed the alert threshold at t...
3,Ghana,12/30/13,3,0.0,Two districts crossed the alert threshold at t...
4,Mali,12/30/13,0,0.0,Two districts crossed the alert threshold at t...


In [ ]:
country_list = []